# Run quantum accurate geometry optimisation on a small molecule

This notebook shows how to geometry optimise a small molecule using RIMP2 accurate gradients.

In [3]:
import os
from glob import glob
from pathlib import Path
import json

import rush

## 0) Setup

In [5]:
# Define our project information
DESCRIPTION = "rush-py qm geo opt notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "qm_geo_opt"]
WORK_DIR = Path.home() / "qdx" / "qm_geo_opt"

In [6]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=True)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys
os.chdir(WORK_DIR)

## 0.1) Initialize our rush client and fetch available module paths

In [7]:
# By using the `build_provider_with_functions` method, we will also build helper functions calling each module
client = await rush.build_provider_with_functions(
    access_token=RUSH_TOKEN, url=RUSH_URL, batch_tags=TAGS
)

In [8]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

## 0.2) Get qm_geo_opt rush module

In [9]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducable
# This will be done automatically if you use the `build_provider_with_functions` method
modules = await client.get_latest_module_paths()

In [10]:
module_name = "qm_geo_opt"
module_path = modules[module_name]
print(module_path)

github:talo/qm_geo_opt/dabb98a630a7c821e2fe2b32ea876c19c8866265#qm_geo_opt_tengu


In [11]:
help(client.qm_geo_opt)

Help on function qm_geo_opt in module rush.provider:

async qm_geo_opt(*args: [<class 'pathlib.Path'>, dict[str, ~T]], target: rush.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH_GPU: 'NIX_SSH_GPU'>, resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=0, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=10, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>]
    geometry optimise molecule with quantum accuracy
    
    Module version: github:talo/qm_geo_opt/dabb98a630a7c821e2fe2b32ea876c19c8866265#qm_geo_opt_tengu
    
    QDX Type Description:
    
        input: @Conformer;
    
        parameters: {
    
        basis_set:string,
    
        conv_threshold:f64,
    
        max_iterations:u32,
    
        use_internal_coords:bool
    
    

# 1) Run Geometry Optimisation

## 1.0) Create input geometry
We will be creating a qdxf conformer object from an unoptimised cyclobutane xyz

In [31]:
def xyz_to_qdxf_topology(xyz_data : str):
  symbols = []
  geometry = []
  for line in xyz_data.splitlines()[2:]:
    symbol, x, y, z = line.split()
    symbols.append(symbol)
    geometry.extend([float(x), float(y), float(z)])
  return {
    "topology": {
      "symbols": symbols,
      "geometry": geometry
    }
  }

def qdxf_topology_to_xyz(qdxf_in : dict, name : str):
  symbols = qdxf_in["topology"]["symbols"]
  geometry = qdxf_in["topology"]["geometry"]
  xyz_data = f"{len(symbols)}\n{name}\n"
  for i in range(len(symbols)):
    symbol = symbols[i]
    x, y, z = geometry[3*i:3*i+3]
    xyz_data += f"  {symbol}"
    for coord in [x, y, z]:
      xyz_data += "     "
      if (coord >= 0):
        xyz_data += " "
      xyz_data += f"{(coord):.4f}"
    xyz_data += "\n"
  return xyz_data

In [18]:
cyclobutane_xyz = '''12
cyclobutane
  C     -0.750      0.452     -0.417
  C     -0.696     -0.588      0.609
  C      0.820     -0.678      0.537
  C      0.892      0.417     -0.428
  H     -1.285      1.273      0.066
  H     -1.328      0.080     -1.263
  H     -1.225     -1.507      0.366
  H     -1.029     -0.162      1.555
  H      1.158     -1.594      0.054
  H      1.310     -0.477      1.488
  H      1.432      0.009     -1.290
  H      1.506      1.236     -0.056
'''

In [19]:
cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

In [20]:
# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
  json.dump(cyclobutane_qdxf, f)

## 1.1) Create geometry optimisation parameters 

In [23]:
params = {
  "use_internal_coords": True,
  "max_iterations": 100,
  "conv_threshold": 1e-4,
  "basis_set": "cc-pVDZ"
}

## 1.1) Call geometry optimisation over input geometry

In [24]:
(geo_opt_out,) = await client.qm_geo_opt(cyclobutane_qdxf_input_file, params, target="NIX_SSH_GPU")
print(await geo_opt_out.get())

2024-01-31 20:35:32,560 - rush - INFO - Argument 952046a3-47a3-46dd-ac83-63356e69146e is now RESOLVING
2024-01-31 20:35:45,885 - rush - INFO - Argument 952046a3-47a3-46dd-ac83-63356e69146e is now ADMITTED
2024-01-31 20:35:58,711 - rush - INFO - Argument 952046a3-47a3-46dd-ac83-63356e69146e is now DISPATCHED
2024-01-31 20:36:04,518 - rush - INFO - Argument 952046a3-47a3-46dd-ac83-63356e69146e is now RUNNING
2024-01-31 20:36:25,505 - rush - INFO - Argument 952046a3-47a3-46dd-ac83-63356e69146e is now AWAITING_UPLOAD
[[{'topology': {'geometry': [-0.67606324, 0.23964196, -0.5981247, -0.7034515, -0.58626497, 0.71332735, 0.8360118, -0.7191075, 0.5885512, 0.81175923, 0.5532161, -0.29659742, -1.3809861, 1.076288, -0.7115693, -0.7614481, -0.42277363, -1.4741267, -1.3121647, -1.5016669, 0.7494691, -0.9682505, 0.054577734, 1.5692675, 1.1001319, -1.6036026, -0.012563882, 1.4435291, -0.71427816, 1.5054237, 1.5178174, 0.62624854, -1.1367724, 0.8981147, 1.4587214, 0.32471547], 'symbols': ['C', 'C', 'C

## 1.2) Check calculation converged

In [25]:
(topologies_out, results_out) = await geo_opt_out.get()
assert(results_out["converged"])

# 2) Visualise pre-optimised and optimised geometries

In [32]:
import py3Dmol

In [33]:
pre_opt_view = py3Dmol.view()
pre_opt_view.addModel(cyclobutane_xyz, "xyz")
pre_opt_view.setStyle({'stick':{}})
pre_opt_view.zoomTo()
pre_opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [36]:
(topologies_out, _) = await geo_opt_out.get()
opt_cyclobutane_xyz = qdxf_topology_to_xyz(topologies_out[-1], "optimised cyclobutane")

opt_view = py3Dmol.view()
opt_view.addModel(opt_cyclobutane_xyz, "xyz", {})
opt_view.setStyle({'stick':{}})
opt_view.zoomTo()
opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol